In [17]:
import sys
sys.path.append('../')
sys.dont_write_bytecode = True
import os
import numpy as np
import scipy.ndimage
from matplotlib.pyplot import *
%matplotlib inline
import numba
sys.path.append('/Users/Michael_Saha/Repositories/')
sys.path.append('./../../../')
from fire_sim import *

import blahb

ImportError: No module named 'blahb.PixelSet'

In [16]:
blahb.PixelSet()

AttributeError: 'module' object has no attribute 'PixelSet'

In [8]:
# Break into classes:
#
#    Landscape -> Environmental variables and fire history
#    Town -> Container for agents to loop over
#    
#    Calibration -> Contains a Landscape and manages the training/evaluation of 
#        fire behaviour against historical fire data.
#
#    ABM -> Container for parameterized Landscape and Towns of peoples.
#        Town (and home) behaviours depend on the Landscape and Town behaviours.

In [11]:
class Town:
    def __init__(self, homes, initial_ds, risk, effect, policy, model):
        """Initialize a Town.
        
        Arguments:
            homes - 2-tuple of numpy.ndarray(size=N)
            initial_ds - numpy.ndarray(size=N, dtpye=float, range=[0,1])
            risk - numpy.ndarray(size=N, dtype=float)
            effect - numpy.ndarray(size=N, dtype=float)
            policy - numpy.ndarray(size=N, dtype=float)
            link - Callable that combines (risk, effect, policy, norms, experience) and returns ds: flaot [0,1]
            
        Notes:
            risk is "realized_risk", which is risk * risk_beta and is static for each agent
            effect is "realized_effect" = effect * effect_beta
            policy is "realized_policy" = policy * policy_beta
        """
        self.homes = homes   # y, x
        self.ds = initial_ds
        self.betas = betas
        self.model = model
    
    @property
    def norms(self):
        return np.mean(self.ds)
    
    
    def update_ds(self):
        model(self.betas, self.risk_effect_pol, self.close_fires)



@numba.jit(nopython=True)
def min_distances(A, B, distances):
    """Find the minimum distance of each pixel in A to any pixel in B."""
    assert A[0].size == A[1].size
    assert B[0].size == B[1].size
    assert A[0].size > 0 and B[0].size > 0
    for i in range(A[0].size):
        distances[i] = euclidean_distance(A[0][i], A[1][i], B[0][0], B[1][0])
        for j in range(B[0].size):
            d = euclidean_distance(A[0][i], A[1][i], B[0][j], B[1][j])
            if d < distances[i]:
                distances[i] = d


class ABM:
    """Coupled Towns and Landscapes"""
    
    def __init__(towns, landscape):
        assert all(all_in_range(town) for town in towns)
        self.landscape = landscape
        self.towns = towns
    
    
    def nearest_fires(self):
        """Find the nearest fire in the past five years."""
        for town in self.towns:
            for fire in zip(self.landscape.past_fires, self.landscape.past_fire_map):
                distance_to_last_fire(town.homes, fire, distances)
    
    def update_ds(self):
        """Update defensible space on the landscape using the thinning algorithm."""
        town[ds] = f(town, landscape, model, betas)
        
    def clear_ds(self):
        pass
        
    def update(self):
        self.update_ds()
        self.clear_ds()

In [13]:
A = np.arange(1000), np.arange(1000)
B = np.arange(100,200), np.arange(100,200)

distance = np.zeros(A[0].size)
%timeit d = min_distances(A, B, distance)
distance

1000 loops, best of 3: 530 µs per loop


array([  141.42135624,   140.00714267,   138.59292911,   137.17871555,
         135.76450199,   134.35028843,   132.93607486,   131.5218613 ,
         130.10764774,   128.69343418,   127.27922061,   125.86500705,
         124.45079349,   123.03657993,   121.62236636,   120.2081528 ,
         118.79393924,   117.37972568,   115.96551211,   114.55129855,
         113.13708499,   111.72287143,   110.30865787,   108.8944443 ,
         107.48023074,   106.06601718,   104.65180362,   103.23759005,
         101.82337649,   100.40916293,    98.99494937,    97.5807358 ,
          96.16652224,    94.75230868,    93.33809512,    91.92388155,
          90.50966799,    89.09545443,    87.68124087,    86.2670273 ,
          84.85281374,    83.43860018,    82.02438662,    80.61017306,
          79.19595949,    77.78174593,    76.36753237,    74.95331881,
          73.53910524,    72.12489168,    70.71067812,    69.29646456,
          67.88225099,    66.46803743,    65.05382387,    63.63961031,
      

In [ ]:
# tuple of numpy arrays
# 